In [1]:
import pandas as pd
import os

from pyspark.sql import SparkSession


StatementMeta(, a4a5a31a-1bb9-46b8-ac98-7d5b1ef9c7c8, 3, Finished, Available)

In [2]:
spark = SparkSession.builder \
    .appName("mi_app") \
    .config("spark.driver.maxResultSize", "24g") \
    .getOrCreate()

StatementMeta(, a4a5a31a-1bb9-46b8-ac98-7d5b1ef9c7c8, 4, Finished, Available)

In [3]:
import os
import pandas as pd

def carga_datos_eda():
    current_directory = os.getcwd()

    # Mostrar el directorio actual
    print("Estás en el directorio:", current_directory)
    desired_directory = '/lakehouse/default/Tables' # directorio en el que deseamos trabajar
    os.chdir(desired_directory) # cambiamos al directorio donde deseamos trabajar
    print("Ahora Estás en el directorio:", desired_directory)
    tablas = os.listdir(desired_directory) # instanciamos en una variable la lista de ficheros del directorio donde estamos

    lista_csv = os.listdir('/lakehouse/default/Files/DATA_PROCESADA') # listamos los ficheros que se encuentran en DATA_PROCESADA

    for tabla in tablas:
        dataframe = f'df_{tabla}.csv'

        # LLamamos a la tabla desde el almacen de datos
        df = spark.sql(f"SELECT * FROM LH_destino_PFH.`{tabla}`")

        if dataframe in lista_csv:
            continue
        elif tabla == 'USERYELP' or tabla == 'ESTADOSGOOGLE':

            # Definir el directorio de salida donde guardar los archivos CSV particionados
            output_path = 'abfss://558991ad-6fdf-463a-b58a-238ddabaa398@onelake.dfs.fabric.microsoft.com/8c2776b5-2722-4851-8d3f-07921cab926e/Files/DATA_PROCESADA'

            # Decide el número de particiones que deseas para la repartición
            num_partitions = 45

            # Reparticiona el DataFrame
            df_repartitioned = df.repartition(num_partitions)

            # Convertir los datos particionados en un DataFrame de pandas
            # df_partitioned = df_repartitioned.toPandas()

            # Especificar la ruta del archivo Parquet para guardar los datos particionados
            parquet_path = f"{output_path}/partitioned_{tabla}.parquet"

            # Guardar el DataFrame particionado como archivos Parquet
            df_repartitioned.write.parquet(parquet_path, mode='overwrite')


            print(f'Se guardaron correctamente los datos particionados de la tabla {tabla} en el directorio {output_path}')

            continue

        # Convertir el DataFrame de Spark en un RDD de Python
        rdd = df.rdd.map(tuple)

        # Instanciamos la data en un objeto dataframe y establecemos el path donde lo vamos a guardar
        df_pandas = pd.DataFrame(rdd.collect(), columns=df.columns)
        path_csv = f'abfss://558991ad-6fdf-463a-b58a-238ddabaa398@onelake.dfs.fabric.microsoft.com/8c2776b5-2722-4851-8d3f-07921cab926e/Files/DATA_PROCESADA/df_{tabla}.csv'
        
        # Guardamos la version en .csv
        df_pandas.to_csv(path_csv, index= False)
        print(f'Se guardo correctamente la tabla {tabla} en el directorio /lakehouse/default/Files/DATA_PROCESADA')


StatementMeta(, a4a5a31a-1bb9-46b8-ac98-7d5b1ef9c7c8, 5, Finished, Available)

In [4]:
# procedemos a la carga de las tablas como archivos al directorio designado

carga_datos_eda()

StatementMeta(, a4a5a31a-1bb9-46b8-ac98-7d5b1ef9c7c8, 6, Finished, Available)

Estás en el directorio: /mnt/var/hadoop/tmp/nm-secondary-local-dir/usercache/trusted-service-user/appcache/application_1713665887647_0001/container_1713665887647_0001_01_000001
Ahora Estás en el directorio: /lakehouse/default/Tables
Se guardo correctamente la tabla BUSINESSYELP en el directorio /lakehouse/default/Files/DATA_PROCESADA
Se guardo correctamente la tabla CENSO en el directorio /lakehouse/default/Files/DATA_PROCESADA
Se guardo correctamente la tabla CHECKINYELP en el directorio /lakehouse/default/Files/DATA_PROCESADA
Se guardaron correctamente los datos particionados de la tabla ESTADOSGOOGLE en el directorio abfss://558991ad-6fdf-463a-b58a-238ddabaa398@onelake.dfs.fabric.microsoft.com/8c2776b5-2722-4851-8d3f-07921cab926e/Files/DATA_PROCESADA
Se guardo correctamente la tabla ESTADOSYABREVIATURAS en el directorio /lakehouse/default/Files/DATA_PROCESADA
Se guardo correctamente la tabla METADATAGOOGLE en el directorio /lakehouse/default/Files/DATA_PROCESADA
Se guardo correctame

In [5]:
#veamos los archivos que existen dentro de la carpeta partitioned_USERYELP dentro de DATA_PROCESADA
ruta_carpeta_USER = '/lakehouse/default/Files/DATA_PROCESADA/partitioned_USERYELP.parquet/'
archivos_USER= os.listdir(ruta_carpeta_USER)
print(archivos_USER)

StatementMeta(, a4a5a31a-1bb9-46b8-ac98-7d5b1ef9c7c8, 7, Finished, Available)

['_SUCCESS', 'part-00000-3c0cbf8d-826a-4cf6-9b5a-df54d05887ee-c000.snappy.parquet', 'part-00001-3c0cbf8d-826a-4cf6-9b5a-df54d05887ee-c000.snappy.parquet', 'part-00002-3c0cbf8d-826a-4cf6-9b5a-df54d05887ee-c000.snappy.parquet', 'part-00003-3c0cbf8d-826a-4cf6-9b5a-df54d05887ee-c000.snappy.parquet', 'part-00004-3c0cbf8d-826a-4cf6-9b5a-df54d05887ee-c000.snappy.parquet', 'part-00005-3c0cbf8d-826a-4cf6-9b5a-df54d05887ee-c000.snappy.parquet', 'part-00006-3c0cbf8d-826a-4cf6-9b5a-df54d05887ee-c000.snappy.parquet', 'part-00007-3c0cbf8d-826a-4cf6-9b5a-df54d05887ee-c000.snappy.parquet', 'part-00008-3c0cbf8d-826a-4cf6-9b5a-df54d05887ee-c000.snappy.parquet', 'part-00009-3c0cbf8d-826a-4cf6-9b5a-df54d05887ee-c000.snappy.parquet', 'part-00010-3c0cbf8d-826a-4cf6-9b5a-df54d05887ee-c000.snappy.parquet', 'part-00011-3c0cbf8d-826a-4cf6-9b5a-df54d05887ee-c000.snappy.parquet', 'part-00012-3c0cbf8d-826a-4cf6-9b5a-df54d05887ee-c000.snappy.parquet', 'part-00013-3c0cbf8d-826a-4cf6-9b5a-df54d05887ee-c000.snappy.par

In [6]:
# Eliminamos fichero _SUCCESS
archivo_a_eliminar = "/lakehouse/default/Files/DATA_PROCESADA/partitioned_USERYELP.parquet/_SUCCESS"

# Intenta eliminar el archivo
try:
    os.remove(archivo_a_eliminar)
    print(f"El archivo {archivo_a_eliminar} ha sido eliminado correctamente.")
except OSError as e:
    print(f"No se pudo eliminar el archivo {archivo_a_eliminar}: {e}")

StatementMeta(, a4a5a31a-1bb9-46b8-ac98-7d5b1ef9c7c8, 8, Finished, Available)

El archivo /lakehouse/default/Files/DATA_PROCESADA/partitioned_USERYELP.parquet/_SUCCESS ha sido eliminado correctamente.


In [7]:
#veamos los archivos que existen dentro de la carpeta partitioned_USERYELP dentro de DATA_PROCESADA
ruta_carpeta_ESTADOS = '/lakehouse/default/Files/DATA_PROCESADA/partitioned_ESTADOSGOOGLE.parquet/'
archivos_ESTADOS= os.listdir(ruta_carpeta_ESTADOS)
print(archivos_ESTADOS)

StatementMeta(, a4a5a31a-1bb9-46b8-ac98-7d5b1ef9c7c8, 9, Finished, Available)

['_SUCCESS', 'part-00000-216e6497-e9e5-4b88-a8fc-d56c0cca770b-c000.snappy.parquet', 'part-00001-216e6497-e9e5-4b88-a8fc-d56c0cca770b-c000.snappy.parquet', 'part-00002-216e6497-e9e5-4b88-a8fc-d56c0cca770b-c000.snappy.parquet', 'part-00003-216e6497-e9e5-4b88-a8fc-d56c0cca770b-c000.snappy.parquet', 'part-00004-216e6497-e9e5-4b88-a8fc-d56c0cca770b-c000.snappy.parquet', 'part-00005-216e6497-e9e5-4b88-a8fc-d56c0cca770b-c000.snappy.parquet', 'part-00006-216e6497-e9e5-4b88-a8fc-d56c0cca770b-c000.snappy.parquet', 'part-00007-216e6497-e9e5-4b88-a8fc-d56c0cca770b-c000.snappy.parquet', 'part-00008-216e6497-e9e5-4b88-a8fc-d56c0cca770b-c000.snappy.parquet', 'part-00009-216e6497-e9e5-4b88-a8fc-d56c0cca770b-c000.snappy.parquet', 'part-00010-216e6497-e9e5-4b88-a8fc-d56c0cca770b-c000.snappy.parquet', 'part-00011-216e6497-e9e5-4b88-a8fc-d56c0cca770b-c000.snappy.parquet', 'part-00012-216e6497-e9e5-4b88-a8fc-d56c0cca770b-c000.snappy.parquet', 'part-00013-216e6497-e9e5-4b88-a8fc-d56c0cca770b-c000.snappy.par

In [8]:
# Eliminamos fichero _SUCCESS
archivo_a_eliminar = "/lakehouse/default/Files/DATA_PROCESADA/partitioned_ESTADOSGOOGLE.parquet/_SUCCESS"

# Intenta eliminar el archivo
try:
    os.remove(archivo_a_eliminar)
    print(f"El archivo {archivo_a_eliminar} ha sido eliminado correctamente.")
except OSError as e:
    print(f"No se pudo eliminar el archivo {archivo_a_eliminar}: {e}")

StatementMeta(, a4a5a31a-1bb9-46b8-ac98-7d5b1ef9c7c8, 10, Finished, Available)

El archivo /lakehouse/default/Files/DATA_PROCESADA/partitioned_ESTADOSGOOGLE.parquet/_SUCCESS ha sido eliminado correctamente.


In [9]:
# mergeamos el archivo fraccionado de la tabla USERYELP
df_unido = None
path_csv = 'abfss://558991ad-6fdf-463a-b58a-238ddabaa398@onelake.dfs.fabric.microsoft.com/8c2776b5-2722-4851-8d3f-07921cab926e/Files/DATA_PROCESADA/df.USERYELP.csv'
archivos_USER = os.listdir(ruta_carpeta_USER)

for file in archivos_USER:

    df = pd.read_parquet(f'/lakehouse/default/Files/DATA_PROCESADA/partitioned_USERYELP.parquet/{file}')
    
    if df_unido is None:
        df_unido = df
        continue
    else:
        df_unido = pd.concat([df_unido, df])
        print(f'Se transformo y concateno {file} a df_unido')
        continue

#Guardamos el archivo de todas las concatenaciones en el lakehouse destino
df_unido.to_csv(path_csv, index=False)
print('se guardo el df_unido en DATA_PROCESADA')

StatementMeta(, a4a5a31a-1bb9-46b8-ac98-7d5b1ef9c7c8, 11, Finished, Available)

Se transformo y concateno part-00001-3c0cbf8d-826a-4cf6-9b5a-df54d05887ee-c000.snappy.parquet a df_unido
Se transformo y concateno part-00002-3c0cbf8d-826a-4cf6-9b5a-df54d05887ee-c000.snappy.parquet a df_unido
Se transformo y concateno part-00003-3c0cbf8d-826a-4cf6-9b5a-df54d05887ee-c000.snappy.parquet a df_unido
Se transformo y concateno part-00004-3c0cbf8d-826a-4cf6-9b5a-df54d05887ee-c000.snappy.parquet a df_unido
Se transformo y concateno part-00005-3c0cbf8d-826a-4cf6-9b5a-df54d05887ee-c000.snappy.parquet a df_unido
Se transformo y concateno part-00006-3c0cbf8d-826a-4cf6-9b5a-df54d05887ee-c000.snappy.parquet a df_unido
Se transformo y concateno part-00007-3c0cbf8d-826a-4cf6-9b5a-df54d05887ee-c000.snappy.parquet a df_unido
Se transformo y concateno part-00008-3c0cbf8d-826a-4cf6-9b5a-df54d05887ee-c000.snappy.parquet a df_unido
Se transformo y concateno part-00009-3c0cbf8d-826a-4cf6-9b5a-df54d05887ee-c000.snappy.parquet a df_unido
Se transformo y concateno part-00010-3c0cbf8d-826a-4cf6

In [10]:
# mergeamos el archivo fraccionado de la tabla ESTADOSGOOGLE
df_unido = None
path_csv = 'abfss://558991ad-6fdf-463a-b58a-238ddabaa398@onelake.dfs.fabric.microsoft.com/8c2776b5-2722-4851-8d3f-07921cab926e/Files/DATA_PROCESADA/df.ESTADOSGOOGLE.csv'
archivos_ESTADOS= os.listdir(ruta_carpeta_ESTADOS)

for file in archivos_ESTADOS:

    df = pd.read_parquet(f'/lakehouse/default/Files/DATA_PROCESADA/partitioned_ESTADOSGOOGLE.parquet/{file}')
    
    if df_unido is None:
        df_unido = df
        continue
    else:
        df_unido = pd.concat([df_unido, df])
        print(f'Se transformo y concateno {file} a df_unido')
        continue

#Guardamos el archivo de todas las concatenaciones en el lakehouse destino
df_unido.to_csv(path_csv, index=False)
print('se guardo el df_unido en DATA_PROCESADA')

StatementMeta(, a4a5a31a-1bb9-46b8-ac98-7d5b1ef9c7c8, 12, Finished, Available)

Se transformo y concateno part-00001-216e6497-e9e5-4b88-a8fc-d56c0cca770b-c000.snappy.parquet a df_unido
Se transformo y concateno part-00002-216e6497-e9e5-4b88-a8fc-d56c0cca770b-c000.snappy.parquet a df_unido
Se transformo y concateno part-00003-216e6497-e9e5-4b88-a8fc-d56c0cca770b-c000.snappy.parquet a df_unido
Se transformo y concateno part-00004-216e6497-e9e5-4b88-a8fc-d56c0cca770b-c000.snappy.parquet a df_unido
Se transformo y concateno part-00005-216e6497-e9e5-4b88-a8fc-d56c0cca770b-c000.snappy.parquet a df_unido
Se transformo y concateno part-00006-216e6497-e9e5-4b88-a8fc-d56c0cca770b-c000.snappy.parquet a df_unido
Se transformo y concateno part-00007-216e6497-e9e5-4b88-a8fc-d56c0cca770b-c000.snappy.parquet a df_unido
Se transformo y concateno part-00008-216e6497-e9e5-4b88-a8fc-d56c0cca770b-c000.snappy.parquet a df_unido
Se transformo y concateno part-00009-216e6497-e9e5-4b88-a8fc-d56c0cca770b-c000.snappy.parquet a df_unido
Se transformo y concateno part-00010-216e6497-e9e5-4b88

In [11]:
tablas= os.listdir('/lakehouse/default/Files/DATA_PROCESADA')
print(tablas)

StatementMeta(, a4a5a31a-1bb9-46b8-ac98-7d5b1ef9c7c8, 13, Finished, Available)

['partitioned_ESTADOSGOOGLE.parquet', 'partitioned_USERYELP.parquet', 'df.ESTADOSGOOGLE.csv', 'df.USERYELP.csv', 'df_BUSINESSYELP.csv', 'df_CENSO.csv', 'df_CHECKINYELP.csv', 'df_ESTADOSYABREVIATURAS.csv', 'df_METADATAGOOGLE.csv', 'df_REVIEWYELP.csv', 'df_TIPYELP.csv', 'df_TRIPADVISOR.csv', 'merged_reviews']


In [12]:
a= pd.read_csv('abfss://558991ad-6fdf-463a-b58a-238ddabaa398@onelake.dfs.fabric.microsoft.com/8c2776b5-2722-4851-8d3f-07921cab926e/Files/DATA_PROCESADA/df_TRIPADVISOR.csv')
a.head()

StatementMeta(, a4a5a31a-1bb9-46b8-ac98-7d5b1ef9c7c8, 14, Finished, Available)

,ID,LOCATION_ID,PUBLISHED_DATE,RATING,TITLE,USERNAME,STATE,NAME,LATITUDE,LONGITUDE,SENTIMENT_ANALYSIS
0,947109079,217164,2024-04-17,5,"Great location, friendly place",Socalblond,California,Bishop,37.36679,-118.39535,2
1,944581119,3335765,2024-03-30,5,RV trip,rkeitha,California,Cloverdale,38.80610,-123.01772,2
2,943655648,13511767,2024-03-24,5,Comfortable & Clean!,travelqueen599,California,Coalinga,36.13702,-120.36289,2
3,943292122,23624111,2024-03-21,5,Relaxing,nadja362,California,Copperopolis,37.98246,-120.64538,2
4,944737064,77546,2024-04-01,5,A Highly recommended location,psalms51warrior,California,Lodi,38.13019,-121.27547,2


In [13]:
b= pd.read_csv('abfss://558991ad-6fdf-463a-b58a-238ddabaa398@onelake.dfs.fabric.microsoft.com/8c2776b5-2722-4851-8d3f-07921cab926e/Files/DATA_PROCESADA/df.ESTADOSGOOGLE.csv')
b.head()

StatementMeta(, a4a5a31a-1bb9-46b8-ac98-7d5b1ef9c7c8, 15, Finished, Available)

,USER_ID,NAME,RATING,GMAP_ID,STATE,DATE,SENTIMENT_ANALYSIS
0,1.052967e+36,Oscar Pérez,2,0x88f5a756f3e2a719:0xb192d571bc5cfee3,Georgia,2019-07-20,2
1,1.035703e+34,Sonya Guzman,2,0x88ee01e47d30de5b:0x5ce9b28668d7cb4b,Georgia,2019-05-27,2
2,1.041607e+36,Jacob Fox,2,0x88ee65ebb8812181:0x467967197bb64da6,Georgia,2012-09-10,2
3,1.020528e+36,Jenine Jones,2,0x88fb1bfb16b5fce1:0x8b43418c511dd372,Georgia,2017-06-12,2
4,1.005355e+36,Kyle Torrey,2,0x88f45afab07b2cd5:0x389b63979a59cd63,Georgia,2020-09-18,2


In [14]:
df = spark.sql("SELECT * FROM LH_destino_PFH.`USERYELP`")
display(df)

StatementMeta(, a4a5a31a-1bb9-46b8-ac98-7d5b1ef9c7c8, 16, Finished, Available)

SynapseWidget(Synapse.DataFrame, 85f87685-5581-47cd-ab20-f547307f40f8)